<a href="https://colab.research.google.com/github/shekhs/tweet-sentiment-analysis/blob/main/Tweets_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing sentiment of tweets
------------------

### Importing the Modules

In [22]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import re
import string
!pip install nltk
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('stopwords')
nltk.download('vader_lexicon')


from collections import Counter

from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
import plotly.express as px

sns.set(style="darkgrid")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Importing the Dataset

In [23]:
link="https://raw.githubusercontent.com/gabrielpreda/covid-19-tweets/master/covid19_tweets.csv"
df = pd.read_csv(link)

##### exploring the data`

In [24]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [25]:
df.shape

(179108, 13)

##### Selecting only required columns

In [26]:
df=df[["user_name","date","text"]]

In [27]:
df.head()

,user_name,date,text
0,ᏉᎥ☻լꂅϮ,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,Time4fisticuffs,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,ethel mertz,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...
4,DIPR-J&K,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...


### change the type of date column, removing the time component, as only date is required.

In [28]:
df.date = pd.to_datetime(df.date).dt.date

### encoding the user name

In [29]:
df.user_name = df.user_name.astype("category")
df.user_name = df.user_name.cat.codes

In [30]:
df.head()

,user_name,date,text
0,89755,2020-07-25,If I smelled the scent of hand sanitizers toda...
1,76403,2020-07-25,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,76147,2020-07-25,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,84572,2020-07-25,@brookbanktv The one gift #COVID19 has give me...
4,18398,2020-07-25,25 July : Media Bulletin on Novel #CoronaVirus...


### Picking out the tweet texts

In [31]:
text = df.text

In [32]:
text

0         If I smelled the scent of hand sanitizers toda...
1         Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2         @diane3443 @wdunlap @realDonaldTrump Trump nev...
3         @brookbanktv The one gift #COVID19 has give me...
4         25 July : Media Bulletin on Novel #CoronaVirus...
                                ...                        
179103    Thanks @IamOhmai for nominating me for the @WH...
179104    2020! The year of insanity! Lol! #COVID19 http...
179105    @CTVNews A powerful painting by Juan Lucena. I...
179106    More than 1,200 students test positive for #CO...
179107    I stop when I see a Stop\n\n@SABCNews\n@Izinda...
Name: text, Length: 179108, dtype: object

### Removing URLs from tweets

In [33]:
rem_url = lambda x:re.sub(r'https\S+','',str(x))

In [34]:
txt = text.apply(rem_url)

In [35]:
txt

0         If I smelled the scent of hand sanitizers toda...
1         Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2         @diane3443 @wdunlap @realDonaldTrump Trump nev...
3         @brookbanktv The one gift #COVID19 has give me...
4         25 July : Media Bulletin on Novel #CoronaVirus...
                                ...                        
179103    Thanks @IamOhmai for nominating me for the @WH...
179104           2020! The year of insanity! Lol! #COVID19 
179105    @CTVNews A powerful painting by Juan Lucena. I...
179106    More than 1,200 students test positive for #CO...
179107    I stop when I see a Stop\n\n@SABCNews\n@Izinda...
Name: text, Length: 179108, dtype: object

### Converting all tweets to lowercase

In [36]:
to_low = lambda x:x.lower()
txt_lr = txt.apply(to_low)

In [37]:
txt_lr

0         if i smelled the scent of hand sanitizers toda...
1         hey @yankees @yankeespr and @mlb - wouldn't it...
2         @diane3443 @wdunlap @realdonaldtrump trump nev...
3         @brookbanktv the one gift #covid19 has give me...
4         25 july : media bulletin on novel #coronavirus...
                                ...                        
179103    thanks @iamohmai for nominating me for the @wh...
179104           2020! the year of insanity! lol! #covid19 
179105    @ctvnews a powerful painting by juan lucena. i...
179106    more than 1,200 students test positive for #co...
179107    i stop when i see a stop\n\n@sabcnews\n@izinda...
Name: text, Length: 179108, dtype: object

### Removing punctuations

In [38]:
rm_punc = lambda x :x.translate(str.maketrans("","",string.punctuation))
txt_np = txt_lr.apply(rm_punc)

In [39]:
txt_np

0         if i smelled the scent of hand sanitizers toda...
1         hey yankees yankeespr and mlb  wouldnt it have...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv the one gift covid19 has give me i...
4         25 july  media bulletin on novel coronavirusup...
                                ...                        
179103    thanks iamohmai for nominating me for the who ...
179104               2020 the year of insanity lol covid19 
179105    ctvnews a powerful painting by juan lucena its...
179106    more than 1200 students test positive for covi...
179107    i stop when i see a stop\n\nsabcnews\nizindaba...
Name: text, Length: 179108, dtype: object

### Removing stopwords

In [40]:
more_stops = ['covid','#coronavirus', '#coronavirusoutbreak', '#coronavirusPandemic', '#covid19', '#covid_19', '#epitwitter', '#ihavecorona', 'amp', 'coronavirus', 'covid19']
stop_words = stopwords.words("English")
stop_words=stop_words+more_stops

OSError: ignored

In [41]:
remove_words = lambda x:" ".join([word for word in x.split() if word not in stop_words])

In [42]:
txt_ns = txt_np.apply(remove_words)

NameError: ignored

### Trimming white spaces

In [43]:
sp_trim = lambda x:x.strip()
txt_ns = txt_ns.apply(sp_trim)

NameError: ignored

In [45]:
txt_ns

NameError: ignored

### create a big list of words out of all the tweets 

In [46]:
word_list = [word for line in txt_ns for word in line.split()]

NameError: ignored

In [47]:
len(word_list)

NameError: ignored

In [ ]:
word_counts = Counter(word_list).most_common(50)
word_counts
word_df = pd.DataFrame(word_counts,columns=["Word","count"])
word_df.head()

In [ ]:
px.histogram(word_df,x="Word",y="count",title="Most Common Words")

### put the Cleaned text in main dataframe

In [ ]:
df.text = txt_ns
df.head()

# Sentiment Analysis 

Getting the polarity scores for each tweet

In [ ]:
sid = SentimentIntensityAnalyzer()
ps =lambda x: sid.polarity_scores(x)

In [ ]:
df["Senti_Score"] = df.text.apply(ps)

In [ ]:
df.head()

In [ ]:
senti_df = pd.DataFrame(data=list(df.Senti_Score))

In [ ]:
senti_df.head()

### Labeling the scores based on the compound polarity value

In [ ]:
labelize = lambda x:"neutral" if x==0 else("positive" if x>0 else "negative")
senti_df["label"] = senti_df.compound.apply(labelize)

In [ ]:
senti_df.head()

In [ ]:
df = df.drop(columns="Senti_Score",axis=1)
df.head()

### let's join two dataframes

In [ ]:
data = df.join(senti_df.label)
data.head()

### Plotting the sentiment score counts

In [ ]:
counts_df = data.label.value_counts().reset_index()
counts_df.head()

In [ ]:
sns.barplot(x="index",y="label",data=counts_df)

In [ ]:
data_agg = data.groupby(["date","label"]).count().reset_index()
data_agg

In [ ]:
px.line(data_agg,x="date",y="text",color="label",title="Daily tweets sentimental analysis")